## A template to show how to setup model training 

In [1]:
from dask_gateway import Gateway
gateway = Gateway()

# close existing clusters
open_clusters = gateway.list_clusters()
print(list(open_clusters))

flag_shut = True 

if flag_shut: 
    if len(open_clusters)>0:
        for c in open_clusters:
            cluster = gateway.connect(c.name)
            cluster.shutdown()  

options = gateway.cluster_options()

#options.worker_memory = 16 # 24 works fine for long term mean, but seasonal needs a bit more
# options.worker_cores = 8

options.environment = dict(
    DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION="1.0"
)

# Create a cluster with those options
cluster = gateway.new_cluster(options)
client = cluster.get_client()
cluster.adapt(15, 40)
#cluster.scale(10)
client

[]


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.286204df8f9943dfa8250e1ce4e4545e/status,


2024-09-05 14:52:33,509 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [2]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [4]:
import sys
sys.path.append('/home/jovyan/mesoscale_buoyancy_param_ML/modules/')

In [5]:
%reload_ext autoreload
%autoreload 2
import datasets

First we **read in the simulation data** from as many experiments of filter scales we need. 
This is handled by `SimulationData` class. At this stage one can define all the extra variables that may be needed for ML, e.g. if we need variables that are non-dim, or rotated, or have a wider stencil etc. 

In [6]:
DT = datasets.SimulationData(simulation_names=['P2L', 'DG'], filter_scales=['50','100','200','400'])
DT.preprocess_simulation_data(window_size=3)

In [7]:
DT.simulation_data

DataTree('None', parent=None)
├── DataTree('P2L')
│   ├── DataTree('50')
│   │       Dimensions:                      (Time: 7200, xh: 150, yh: 200, zl: 2, Xn: 3,
│   │                                         Yn: 3, zi: 3)
│   │       Coordinates:
│   │         * Time                         (Time) object 58kB 0001-01-11 00:00:00 ... 0...
│   │         * xh                           (xh) float64 1kB 4.0 12.0 ... 1.196e+03
│   │         * yh                           (yh) float64 2kB 4.0 12.0 ... 1.596e+03
│   │         * zl                           (zl) float64 16B 1.031e+03 1.033e+03
│   │         * zi                           (zi) float64 24B 1.03e+03 1.032e+03 1.034e+03
│   │       Dimensions without coordinates: Xn, Yn
│   │       Data variables: (12/40)
│   │           dudx_widened                 (Time, zl, yh, xh, Xn, Yn) float64 31GB dask.array<chunksize=(36, 2, 199, 149, 3, 3), meta=np.ndarray>
│   │           dvdx_widened                 (Time, zl, yh, xh, Xn, Yn) float64 31GB dask.array<chunksize=(36, 2, 199, 149, 3, 3), meta=np.ndarray>
│   │           dudy_widened                 (Time, zl, yh, xh, Xn, Yn) float64 31GB dask.array<chunksize=(36, 2, 199, 149, 3, 3), meta=np.ndarray>
│   │           dvdy_widened                 (Time, zl, yh, xh, Xn, Yn) float64 31GB dask.array<chunksize=(36, 2, 199, 149, 3, 3), meta=np.ndarray>
│   │           dhdx_widened                 (Time, zl, yh, xh, Xn, Yn) float64 31GB dask.array<chunksize=(36, 2, 199, 149, 3, 3), meta=np.ndarray>
│   │           dhdy_widened                 (Time, zl, yh, xh, Xn, Yn) float64 31GB dask.array<chunksize=(36, 2, 199, 149, 3, 3), meta=np.ndarray>
│   │           ...                           ...
│   │           dvdx_widened_rotated_nondim  (Time, zl, yh, xh, Xn, Yn) float64 31GB dask.array<chunksize=(36, 2, 199, 149, 3, 3), meta=np.ndarray>
│   │           dvdy_widened_rotated_nondim  (Time, zl, yh, xh, Xn, Yn) float64 31GB dask.array<chunksize=(36, 2, 199, 149, 3, 3), meta=np.ndarray>
│   │           dhdx_widened_rotated_nondim  (Time, zl, yh, xh, Xn, Yn) float64 31GB dask.array<chunksize=(36, 2, 199, 149, 3, 3), meta=np.ndarray>
│   │           dhdy_widened_rotated_nondim  (Time, zl, yh, xh, Xn, Yn) float64 31GB dask.array<chunksize=(36, 2, 199, 149, 3, 3), meta=np.ndarray>
│   │           uphp_rotated_nondim          (Time, zl, yh, xh) float64 3GB dask.array<chunksize=(36, 2, 199, 149), meta=np.ndarray>
│   │           vphp_rotated_nondim          (Time, zl, yh, xh) float64 3GB dask.array<chunksize=(36, 2, 199, 149), meta=np.ndarray>
│   │       Attributes:
│   │           simulation_name:  P2L
│   │           filter_scale:     50
│   │           source:           gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/r...
│   │           description:      Dataset for P2L experiment at 50 km resolution
│   ├── DataTree('100')
│   │       Dimensions:                      (Time: 7200, xh: 60, yh: 80, zl: 2, Xn: 3,
│   │                                         Yn: 3, zi: 3)
│   │       Coordinates:
│   │         * Time                         (Time) object 58kB 0001-01-11 00:00:00 ... 0...
│   │         * xh                           (xh) float64 480B 10.0 30.0 ... 1.19e+03
│   │         * yh                           (yh) float64 640B 10.0 30.0 ... 1.59e+03
│   │         * zl                           (zl) float64 16B 1.031e+03 1.033e+03
│   │         * zi                           (zi) float64 24B 1.03e+03 1.032e+03 1.034e+03
│   │       Dimensions without coordinates: Xn, Yn
│   │       Data variables: (12/40)
│   │           dudx_widened                 (Time, zl, yh, xh, Xn, Yn) float64 5GB dask.array<chunksize=(36, 2, 79, 59, 3, 3), meta=np.ndarray>
│   │           dvdx_widened                 (Time, zl, yh, xh, Xn, Yn) float64 5GB dask.array<chunksize=(36, 2, 79, 59, 3, 3), meta=np.ndarray>
│   │           dudy_widened                 (Time, zl, yh, xh, Xn, Yn) float64 5GB dask.array<chunksize=(36, 2, 79, 59, 3, 3), meta=np.ndarray>
│  

Next we **create the dataset that is needed for ML training/testing**. In addition to choosing the variables that we use as input and output for the training, at this stage we also make choices about:
- A time selection is done to choose what datarange to act on, this can be used to distinguish training and testing data (which for now would have to be created as separate objects of this class).
- how to do masking of variables.
- if datasets should be sub-sampled to create equal data weights.
- which dims are stacked together to become samlples.
- dropping of NANs
- randominzed data picking
- concatenation along scales and experiments 

In [8]:
ML_DT_train = datasets.MLDataset(simulation_data=DT,
                                 input_variables=['dudx_widened_rotated_nondim',
                                                  'dvdx_widened_rotated_nondim',
                                                  'dudy_widened_rotated_nondim',
                                                  'dvdy_widened_rotated_nondim',
                                                  'dhdx_widened_rotated_nondim',
                                                  'dhdy_widened_rotated_nondim',
                                                  'filter_scale'],
                                 output_variables=['uphp_rotated_nondim','vphp_rotated_nondim'], 
                                time_range=slice(-2000, None))

In [9]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0037 seconds
Time subsampling took: 0.0459 seconds
Horizontal subsampling took: 0.0252 seconds
h_mask_ml_variables took: 0.2058 seconds
stack_physical_dimensions took: 1.1356 seconds
will load upto: 8.2502 gb
load took: 534.6314 seconds
drop_nans took: 4.5002 seconds
Seed set as:42
randomize_along_points took: 5.8531 seconds
Picked: 936000points
pick_uniform_points took: 0.0074 seconds
concat_datatree_nodes took: 1.1233 seconds
randomize_concatenated_ml_dataset took: 3.6017 seconds
generate_batches took: 0.0002 seconds
CPU times: user 37 s, sys: 6.84 s, total: 43.8 s
Wall time: 9min 11s


In [12]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0036 seconds
Time subsampling took: 0.0461 seconds
Horizontal subsampling took: 0.0245 seconds
h_mask_ml_variables took: 0.1897 seconds
stack_physical_dimensions took: 0.3629 seconds
will load upto: 4.1251 gb
load took: 176.7148 seconds
drop_nans took: 2.0740 seconds
Seed set as:42
randomize_along_points took: 2.7413 seconds
Picked: 468000points
pick_uniform_points took: 0.0074 seconds
concat_datatree_nodes took: 0.5295 seconds
randomize_concatenated_ml_dataset took: 1.7034 seconds
generate_batches took: 0.0002 seconds
CPU times: user 15.4 s, sys: 3.46 s, total: 18.8 s
Wall time: 3min 4s


In [10]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0036 seconds
Time subsampling took: 0.0274 seconds
Horizontal subsampling took: 0.0202 seconds
h_mask_ml_variables took: 0.1706 seconds
stack_physical_dimensions took: 0.1068 seconds
will load upto: 0.41251 gb
load took: 29.8579 seconds
drop_nans took: 0.2433 seconds
Seed set as:42
randomize_along_points took: 0.3033 seconds
Picked: 46800points
pick_uniform_points took: 0.0072 seconds
concat_datatree_nodes took: 0.0736 seconds
randomize_concatenated_ml_dataset took: 0.1610 seconds
generate_batches took: 0.0002 seconds
CPU times: user 6.98 s, sys: 660 ms, total: 7.64 s
Wall time: 31 s


In [ ]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0055 seconds
Time subsampling took: 0.0699 seconds
Horizontal subsampling took: 0.0308 seconds
h_mask_ml_variables took: 0.5936 seconds
stack_physical_dimensions took: 1.1497 seconds
will load upto: 12.3753 gb
load took: 744.3908 seconds
drop_nans took: 6.7816 seconds
Seed set as:42
randomize_along_points took: 8.6521 seconds
Picked: 1404000points
pick_uniform_points took: 0.0277 seconds
concat_datatree_nodes took: 1.4188 seconds


In [ ]:
2+2

In [ ]:
%%time
ML_DT_train.create_ML_variables() 
# this one crashed 

choose_ml_variables took: 0.0041 seconds
Time subsampling took: 0.0669 seconds
Horizontal subsampling took: 0.0260 seconds
h_mask_ml_variables took: 1.1050 seconds
stack_physical_dimensions took: 0.9060 seconds
will load upto: 12.3753 gb
load took: 505.2955 seconds


In [8]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0036 seconds
Time subsampling took: 0.0319 seconds
Horizontal subsampling took: 0.0223 seconds
h_mask_ml_variables took: 0.1789 seconds
stack_physical_dimensions took: 0.2654 seconds
will load: 2.06255 gb
load took: 90.4991 seconds
drop_nans took: 1.0958 seconds
Seed set as:42
randomize_along_points took: 1.4094 seconds
Picked: 234000points
pick_uniform_points took: 0.0071 seconds
concat_datatree_nodes took: 0.2746 seconds
randomize_concatenated_ml_dataset took: 0.8231 seconds
generate_batches took: 0.0002 seconds
CPU times: user 2min 23s, sys: 45.9 s, total: 3min 9s
Wall time: 1min 34s


In [16]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0037 seconds
Time subsampling took: 0.0272 seconds
Horizontal subsampling took: 0.0203 seconds
h_mask_ml_variables took: 0.1724 seconds
stack_physical_dimensions took: 0.1029 seconds
will load: 0.41251 gb
load took: 20.4040 seconds
drop_nans took: 0.1909 seconds
Seed set as:42
randomize_along_points took: 0.2504 seconds
Picked: 46800points
pick_uniform_points took: 1.6472 seconds
concat_datatree_nodes took: 0.0364 seconds
randomize_concatenated_ml_dataset took: 0.1373 seconds
generate_batches took: 0.0002 seconds
CPU times: user 37.2 s, sys: 10.1 s, total: 47.3 s
Wall time: 23 s


In [22]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0037 seconds
Time subsampling took: 0.0401 seconds
Horizontal subsampling took: 0.0205 seconds
h_mask_ml_variables took: 0.1735 seconds
stack_physical_dimensions took: 0.0764 seconds
load took: 7.0459 seconds
drop_nans took: 0.0494 seconds
Seed set as:42
randomize_along_points took: 0.0521 seconds
Picked: 9360points
pick_uniform_points took: 0.0068 seconds
concat_datatree_nodes took: 0.0202 seconds
randomize_concatenated_ml_dataset took: 0.0257 seconds
generate_batches took: 0.0002 seconds
CPU times: user 11.2 s, sys: 3.21 s, total: 14.4 s
Wall time: 7.51 s


In [20]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0035 seconds
Time subsampling took: 0.0387 seconds
Horizontal subsampling took: 0.0211 seconds
h_mask_ml_variables took: 0.1781 seconds
stack_physical_dimensions took: 0.0813 seconds
drop_nans took: 37.5595 seconds
Seed set as:42
randomize_along_points took: 1.9506 seconds
Picked: 9360points
pick_uniform_points took: 0.2531 seconds
concat_datatree_nodes took: 0.1103 seconds
load took: 0.0000 seconds
randomize_concatenated_ml_dataset took: 3.4438 seconds
generate_batches took: 0.0001 seconds
CPU times: user 35.4 s, sys: 7.87 s, total: 43.3 s
Wall time: 43.6 s


In [18]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0034 seconds
Time subsampling took: 0.0296 seconds
Horizontal subsampling took: 0.0201 seconds
h_mask_ml_variables took: 0.1680 seconds
stack_physical_dimensions took: 0.0730 seconds
drop_nans took: 38.5704 seconds
Seed set as:42
randomize_along_points took: 0.9353 seconds
Picked: 9360points
pick_uniform_points took: 1.6452 seconds
concat_datatree_nodes took: 0.1192 seconds
load took: 26.5630 seconds
randomize_concatenated_ml_dataset took: 3.6859 seconds
generate_batches took: 0.0001 seconds
CPU times: user 1min 5s, sys: 13.6 s, total: 1min 18s
Wall time: 1min 11s


In [16]:
%%time
ML_DT_train.create_ML_variables()

choose_ml_variables took: 0.0035 seconds
Time subsampling took: 0.1222 seconds
Horizontal subsampling took: 0.0205 seconds
h_mask_ml_variables took: 0.1742 seconds
stack_physical_dimensions took: 0.0745 seconds
drop_nans took: 39.6761 seconds
Seed set as:42
randomize_along_points took: 0.4738 seconds
Picked: 4680points
pick_uniform_points took: 0.1343 seconds
concat_datatree_nodes took: 0.0654 seconds
load took: 15.4352 seconds
randomize_concatenated_ml_dataset took: 1.2477 seconds
generate_batches took: 0.0001 seconds
CPU times: user 50.2 s, sys: 12.2 s, total: 1min 2s
Wall time: 57.4 s


In [12]:
ML_DT_train.ml_dataset

DataTree('None', parent=None)
├── DataTree('P2L')
│   ├── DataTree('50')
│   │       Dimensions:                      (Xn: 3, Yn: 3, points: 15444)
│   │       Coordinates:
│   │         * points                       (points) object 124kB MultiIndex
│   │         * Time                         (points) object 124kB 0197-08-20 00:00:00 .....
│   │         * xh                           (points) float64 124kB 772.0 836.0 ... 580.0
│   │         * yh                           (points) float64 124kB 1.028e+03 ... 132.0
│   │         * zl                           (points) float64 124kB 1.033e+03 ... 1.031e+03
│   │       Dimensions without coordinates: Xn, Yn
│   │       Data variables:
│   │           dudx_widened_rotated_nondim  (Xn, Yn, points) float64 1MB -0.06218 ... -0...
│   │           dvdx_widened_rotated_nondim  (Xn, Yn, points) float64 1MB 0.07973 ... 0.3175
│   │           dudy_widened_rotated_nondim  (Xn, Yn, points) float64 1MB 0.001858 ... 0....
│   │           dvdy_widened_rotated_nondim  (Xn, Yn, points) float64 1MB 0.0797 ... 0.07876
│   │           dhdx_widened_rotated_nondim  (Xn, Yn, points) float64 1MB 0.3483 ... 0.4096
│   │           dhdy_widened_rotated_nondim  (Xn, Yn, points) float64 1MB 0.2071 ... 0.01862
│   │           filter_scale                 (points) float64 124kB 5e+04 5e+04 ... 5e+04
│   │           uphp_rotated_nondim          (points) float64 124kB -6.264e-06 ... -1.197...
│   │           vphp_rotated_nondim          (points) float64 124kB 1.441e-05 ... 4.279e-06
│   │           h_mask                       (points) bool 15kB True True True ... True True
│   │       Attributes:
│   │           simulation_name:  P2L
│   │           filter_scale:     50
│   │           source:           gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/r...
│   │           description:      Dataset for P2L experiment at 50 km resolution
│   ├── DataTree('100')
│   │       Dimensions:                      (Xn: 3, Yn: 3, points: 15444)
│   │       Coordinates:
│   │         * points                       (points) object 124kB MultiIndex
│   │         * Time                         (points) object 124kB 0197-11-18 00:00:00 .....
│   │         * xh                           (points) float64 124kB 650.0 730.0 ... 90.0
│   │         * yh                           (points) float64 124kB 1.21e+03 330.0 ... 970.0
│   │         * zl                           (points) float64 124kB 1.031e+03 ... 1.031e+03
│   │       Dimensions without coordinates: Xn, Yn
│   │       Data variables:
│   │           dudx_widened_rotated_nondim  (Xn, Yn, points) float64 1MB 0.08512 ... 0.0...
│   │           dvdx_widened_rotated_nondim  (Xn, Yn, points) float64 1MB 0.07189 ... 0.2547
│   │           dudy_widened_rotated_nondim  (Xn, Yn, points) float64 1MB -0.1613 ... 0.0...
│   │           dvdy_widened_rotated_nondim  (Xn, Yn, points) float64 1MB -0.08017 ... -0...
│   │           dhdx_widened_rotated_nondim  (Xn, Yn, points) float64 1MB 0.2313 ... 0.3475
│   │           dhdy_widened_rotated_nondim  (Xn, Yn, points) float64 1MB 0.08938 ... 0.0...
│   │           filter_scale                 (points) float64 124kB 1e+05 1e+05 ... 1e+05
│   │           uphp_rotated_nondim          (points) float64 124kB 1.022e-05 ... 1.731e-05
│   │           vphp_rotated_nondim          (points) float64 124kB 9.395e-06 ... -3.002e-05
│   │           h_mask                       (points) bool 15kB True True True ... True True
│   │       Attributes:
│   │           simulation_name:  P2L
│   │           filter_scale:     100
│   │           source:           gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/r...
│   │           description:      Dataset for P2L experiment at 100 km resolution
│   ├── DataTree('200')
│   │       Dimensions:                      (Xn: 3, Yn: 3, points: 15444)
│   │       Coordinates:
│   │         * points                       (points) object 124kB MultiIndex
│   │         * Time                         (points) objec

In [10]:
ML_DT_train.concatenated_ml_dataset.nbytes/1e9

0.61405344

In [9]:
ML_DT_train.concatenated_ml_dataset

<xarray.Dataset> Size: 67MB
Dimensions:                      (Xn: 3, Yn: 3, points: 134784)
Coordinates:
  * points                       (points) object 1MB MultiIndex
  * Time                         (points) object 1MB 0001-02-10 00:00:00 ... ...
  * xh                           (points) float64 1MB 1.092e+03 250.0 ... 7.975
  * yh                           (points) float64 1MB 1.412e+03 ... 43.14
  * zl                           (points) float64 1MB 1.031e+03 ... 1.036e+03
Dimensions without coordinates: Xn, Yn
Data variables:
    dudx_widened_rotated_nondim  (Xn, Yn, points) float64 10MB dask.array<chunksize=(3, 3, 1), meta=np.ndarray>
    dvdx_widened_rotated_nondim  (Xn, Yn, points) float64 10MB dask.array<chunksize=(3, 3, 1), meta=np.ndarray>
    dudy_widened_rotated_nondim  (Xn, Yn, points) float64 10MB dask.array<chunksize=(3, 3, 1), meta=np.ndarray>
    dvdy_widened_rotated_nondim  (Xn, Yn, points) float64 10MB dask.array<chunksize=(3, 3, 1), meta=np.ndarray>
    dhdx_widened_rotated_nondim  (Xn, Yn, points) float64 10MB dask.array<chunksize=(3, 3, 1), meta=np.ndarray>
    dhdy_widened_rotated_nondim  (Xn, Yn, points) float64 10MB dask.array<chunksize=(3, 3, 1), meta=np.ndarray>
    filter_scale                 (points) float64 1MB dask.array<chunksize=(1,), meta=np.ndarray>
    uphp_rotated_nondim          (points) float64 1MB dask.array<chunksize=(1,), meta=np.ndarray>
    vphp_rotated_nondim          (points) float64 1MB dask.array<chunksize=(1,), meta=np.ndarray>
    h_mask                       (points) bool 135kB dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    simulation_name:  P2L
    filter_scale:     50
    source:           gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/r...
    description:      Dataset for P2L experiment at 50 km resolution